# Capstone Project Toronto with Co-ordinates

### Libraries imported

In [2]:
import pandas as pd
import numpy as np
import folium # map rendering library
import requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pd.set_option('display.max_colwidth', 0)
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
tables = pd.read_html(link,header=0)[0]
tables.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
tables.shape

(180, 3)

## Removing Null Values

In [5]:

# Renaming the columns
tables.columns = ['PostalCode', 'Borough', 'Neighborhood']
# Extracting Table that has an assigned borrow
table_b_a = tables[tables['Borough'] != 'Not assigned']
# Extracting Table that has not assigned borrow
table_b_na = tables[tables['Borough'] == 'Not assigned']

In [6]:
print(table_b_a.shape)
print(table_b_na.shape)

(103, 3)
(77, 3)


In [7]:

# Group by postcode
table_grouped = table_b_a.groupby('PostalCode')
#  These are the keys by which the table is grouped
table_grouped.groups.keys()

dict_keys(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H', 'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B', 'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B', 'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N', 'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A', 'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M', 'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A', 'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M', 'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W', 'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W'])

## Creating New Dataframe

In [8]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

## Loading of Data

In [9]:

def add_comma_neighbor(grp):
    '''
    This function return the borough as well as Neighbor hood for each group combined or joined with comma.
    '''
    return table_grouped.get_group(grp)['Borough'].iloc[0], ", ".join(table_grouped.get_group(grp)['Neighborhood'])

In [10]:
# Looping for each group and appending the result to our empty neighborhoods dataframe just created
for grp in table_grouped.groups.keys():
    borough, neighbor = add_comma_neighbor(grp)
    neighborhoods = neighborhoods.append({'PostalCode' : grp,
                                          'Borough': borough,
                                          'Neighborhood': neighbor
                                         }, ignore_index=True)
neighborhoods

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens


# Part -2 of adding cordinates 

In [11]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Downloading Data

In [12]:
import pandas as pd
df = pd.read_csv("https://cocl.us/Geospatial_data")

In [13]:
df.columns = ['PostalCode', 'Latitude', 'Longitude']
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging Data

In [15]:
neigh_latlong = neighborhoods.merge(df, how='inner', left_on=['PostalCode'], right_on=['PostalCode'])
neigh_latlong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
